In [1]:
import torch
from torch.utils.data import Dataset
import h5py
from PIL import Image
import numpy as np
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torchvision.models.resnet import resnet50
import pandas as pd
from torch.utils.data import Subset
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm

In [2]:
class CharDataset(Dataset):
    def __init__(self, transform=None, include_font_label=True,include_char_label=False, h5_file_path='/content/drive/MyDrive/SynthText.h5'):
        all_text = ''
        self.transform = transform
        self.db = h5py.File(h5_file_path, 'r')
        im_names = list(self.db['data'].keys())
        self.chars_files_names, self.char_index_within_file = [], []
        for image_filename in im_names:
            num_of_characters = self.db['data'][image_filename].attrs['charBB'].shape[-1]
            text = self.db['data'][image_filename].attrs['txt']
            actual_text = ''.join([word.decode('UTF-8') for word in text])
            all_text += actual_text
            for char_index in range(num_of_characters):
                self.chars_files_names.append(image_filename)
                self.char_index_within_file.append(char_index)
        self.fonts_labels = ['Raleway','Open Sans','Ubuntu Mono','Russo One','Alex Brush','Michroma','Roboto']
        self.include_font_label = include_font_label
        self.include_char_label = include_char_label
        self.all_text = all_text
        
    def __getitem__(self, idx):
        
        image_filename = self.chars_files_names[idx]
        char_image_index = self.char_index_within_file[idx]
        img = self.db['data'][image_filename][:]
        character_bounding_box = self.db['data'][image_filename].attrs['charBB'][:, :, char_image_index]
        x0 = min(character_bounding_box[0])
        x1 = max(character_bounding_box[0])
        y0 = min(character_bounding_box[1])
        y1 = max(character_bounding_box[1])
        (x0,y0,x1,y1) = (round(x0),round(y0),round(x1),round(y1))
        y_max,x_max,_ = img.shape
        x0 = min(x_max,max(0,x0))
        x1 = min(x_max, max(0, x1))
        y0 = min(y_max, max(0, y0))
        y1 = min(y_max, max(0, y1))
        cut_image = img[y0:y1, x0:x1, :]
        pil_image = Image.fromarray(cut_image.astype(np.uint8))
        if self.transform is not None:
            pil_image = self.transform(pil_image)
        if self.include_font_label:
            font_name = self.db['data'][image_filename].attrs['font'][char_image_index].decode('UTF-8')
            font_label = self.fonts_labels.index(font_name)
            return pil_image,font_label
        if not self.include_char_label:
            raise ValueError('One of include labels (char or font) has to be True')
        data_dict = {'image': pil_image, 'char': self.all_text[idx], 'file_name': self.chars_files_names[idx]}
        try:
            font_name = self.db['data'][image_filename].attrs['font'][char_image_index].decode('UTF-8')
            font_label = self.fonts_labels.index(font_name)
            data_dict['font_label'] = font_label
        except:
            pass
        return data_dict
        
            
        return pil_image
    def __len__(self):
        return len(self.char_index_within_file)

In [3]:
def train_net(net,l2_regularization=0,learning_rate=10**-4,fold_index=0,epochs=90):


    batch_size = 8
    image_size = (224,224)
    device = 'cuda:0'
    transform = transforms.Compose(
        [transforms.Resize(image_size),transforms.ToTensor(),
         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),transforms.RandomRotation(degrees=(-10,10))])
    
    char_data = CharDataset(transform=transform)
    
    
    
    
    dataset_size = len(char_data)
    validation_size = round(dataset_size*0.2)
    validation_examples_indices = [x for x in range(fold_index*validation_size,(fold_index+1)*validation_size)]
    train_set = Subset(char_data, [x for x in range(0,dataset_size) if x not in validation_examples_indices])
    valid_set = Subset(char_data, [x for x in range(0,dataset_size) if x in validation_examples_indices])
    
    
    trainloader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                              shuffle=True, num_workers=8)
    validation_loader = torch.utils.data.DataLoader(valid_set, batch_size=batch_size,
                                              shuffle=False, num_workers=2)
    
    
    criterion = nn.CrossEntropyLoss()
    
    
    @torch.no_grad()
    def get_accuracy_on_validation(net=net, dataLoader=validation_loader):
        net.eval()
        seen_examples = 0.0
        num_correct_examples = 0
        for (i, data) in enumerate(dataLoader, 0):
            inputs, labels = data
            inputs , labels = inputs.to(device) , labels.to(device)
            outputs = net(inputs)
            num_correct_examples += (outputs.argmax(dim=1) == labels).sum().item()
            seen_examples += inputs.size()[0]
        net.train(True)
        return num_correct_examples/seen_examples
    
    
    
    optimizer = optim.Adam(net.parameters(), lr=learning_rate,weight_decay=l2_regularization)
    
    best_validation = 0.0
    for epoch in range(epochs):  # loop over the dataset multiple times
        print(f'at epoch {epoch}')
        running_loss = 0.0
        for i, data in enumerate(tqdm(trainloader), 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device) #Switch to GPU!
            # zero the parameter gradients
            optimizer.zero_grad()
    
            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward() #calculate the gradient
            optimizer.step() #update the net's parameters
    
            # print statistics
            running_loss += loss.item()
            if i % 2000 == 1999:    # print every 2000 mini-batches
                print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
                running_loss = 0.0
                accuracy_on_validation = get_accuracy_on_validation()
                if best_validation<accuracy_on_validation:
                    best_validation = accuracy_on_validation
                    torch.save(net.state_dict(),f'{best_validation}_fold{fold_index}.net')
                print(f'accuracy on validation is {accuracy_on_validation}')
    
    print('Finished Training')
    return get_accuracy_on_validation()

In [5]:
for fold_index in range(0,5):
  net = resnet50(pretrained=True,num_classes=1000).to(device)
  train_net(net=net,l2_regularization=10**-12,learning_rate=10**-4,epochs=20,fold_index=fold_index)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


at epoch 0


  0%|          | 0/2820 [00:05<?, ?it/s]


RuntimeError: ignored

In [ ]:
our_result = pd.DataFrame(columns=['image','char','Raleway','Open Sans','Roboto','Ubuntu Mono','Michroma','Alex Brush','Russo One'])
device = 'cuda:0'
image_size = (224,224)

In [ ]:
transform = transforms.Compose(
    [transforms.Resize(image_size),transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

test_char_data = CharDataset(transform=transform, include_font_label=False,include_char_label=True,h5_file_path='/content/drive/MyDrive/SynthText_test.h5')
test_load = torch.utils.data.DataLoader(test_char_data, batch_size=1,
                                          shuffle=False, num_workers=8)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torchvision.models.resnet import resnet50
import pandas as pd
from torch.utils.data import Subset
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm
import os
our_result = pd.DataFrame(columns=['image','char','Raleway','Open Sans','Roboto','Ubuntu Mono','Michroma','Alex Brush','Russo One'])
device = 'cuda:0'
NETS_FOLDER = '/content/drive/MyDrive'
nets_filenames = [x for x in os.listdir(NETS_FOLDER) if x[-len('.net'):]=='.net']
image_size = (224,224)

nets=[]
print(f'nets available are {nets_filenames}')
for net_filename in nets_filenames:
    net = resnet50(pretrained=True,num_classes=1000).to(device)
    net.load_state_dict(torch.load(os.path.join(NETS_FOLDER,net_filename)))
    net.eval()
    nets.append(net)



transform = transforms.Compose(
    [transforms.Resize(image_size),transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

test_char_data = CharDataset(transform=transform, include_font_label=False,include_char_label=True,h5_file_path='/content/drive/MyDrive/SynthText_test.h5')
test_load = torch.utils.data.DataLoader(test_char_data, batch_size=1,
                                          shuffle=False, num_workers=8)

@torch.no_grad()
def save_test_estimation(dataLoader=test_load):
    global our_result
    seen_examples = 0.0
    num_correct_examples = 0
    for (i, data) in enumerate(tqdm(dataLoader), 0):
        inputs, image_filename,char = data['image'],data['file_name'],data['char']
        labels = data['font_label'] if 'font_label' in data else None
        inputs  = inputs.to(device)
        probabilities = torch.zeros(1,7).to(device)
        for net in nets:
            outputs = net(inputs)[:,0:7]
            probabilities += torch.nn.Softmax(dim=1)(outputs)
        chosen_font_index = probabilities.argmax(dim=1).item()
        chosen_font_name = test_char_data.fonts_labels[chosen_font_index]
        save_data = {'image':image_filename[0], 'char':char[0], 'Raleway':0, 'Open Sans':0, 'Roboto':0, 'Ubuntu Mono':0, 'Michroma':0, 'Alex Brush':0, 'Russo One':0}
        save_data[chosen_font_name] = 1
        our_result = our_result.append(save_data,ignore_index=True)
        if labels is not None:
            labels = labels.to(device)
            num_correct_examples += (probabilities.argmax(dim=1) == labels).sum().item()
        seen_examples += inputs.size()[0]
        #print(num_correct_examples/seen_examples*100)
    return num_correct_examples/seen_examples
save_test_estimation()
our_result.to_csv('/content/drive/MyDrive/our_result_on_training_set.csv')

for fold_index in range(0,5):
  net = resnet50(pretrained=True,num_classes=1000).to(device)
  train_net(net=net,l2_regularization=10**-12,learning_rate=10**-4,epochs=50,fold_index=fold_index)


In [ ]:
!cat /content/drive/MyDrive/our_result_on_test_set.csv |wc -l

56166
